In [1]:
%pylab inline
import pandas as pd
import os

Populating the interactive namespace from numpy and matplotlib


In [2]:
path = '"../Final/IRI/"'
files = !find $path -name Delivery_Stores

In [3]:
files

['../Final/IRI/Year1/External/yogurt/Delivery_Stores',
 '../Final/IRI/Year10/External/yogurt/Delivery_Stores',
 '../Final/IRI/Year11/External/yogurt/Delivery_Stores',
 '../Final/IRI/Year2/External/yogurt/Delivery_Stores',
 '../Final/IRI/Year3/External/yogurt/Delivery_Stores',
 '../Final/IRI/Year4/External/yogurt/Delivery_Stores',
 '../Final/IRI/Year5/External/yogurt/Delivery_Stores',
 '../Final/IRI/Year6/External/yogurt/Delivery_Stores',
 '../Final/IRI/Year7/External/yogurt/Delivery_Stores',
 '../Final/IRI/Year8/External/yogurt/Delivery_Stores',
 '../Final/IRI/Year9/External/yogurt/Delivery_Stores']

In [4]:
#check if headers differ accross files
listheaders = list()
rowcounts = list()
i = 0
print 'Rows:\t\t', 'Filename:\t\t', 'Year:'
for x in files:
#     x_quoted = path[:-1]+x+'"'
    i += 1
    header = !head -1 $x
    rc = !wc -l $x
    rowcounts.append((int(rc[0].split()[0]), x, i))
    print rc[0].split()[0], '\t', x, '\t', i
    listheaders.append(header[0].strip())

headers = set(listheaders)
print '\nDistinct Headers found:', len(set(listheaders)), 
print '\n', list(headers)
print '\nTotal rows:', sum([x[0] for x in rowcounts]) - len(rowcounts)
print 'Total files:', len(rowcounts)

Rows:		Filename:		Year:
2054 	../Final/IRI/Year1/External/yogurt/Delivery_Stores 	1
2039 	../Final/IRI/Year10/External/yogurt/Delivery_Stores 	2
1923 	../Final/IRI/Year11/External/yogurt/Delivery_Stores 	3
2165 	../Final/IRI/Year2/External/yogurt/Delivery_Stores 	4
2047 	../Final/IRI/Year3/External/yogurt/Delivery_Stores 	5
2022 	../Final/IRI/Year4/External/yogurt/Delivery_Stores 	6
2091 	../Final/IRI/Year5/External/yogurt/Delivery_Stores 	7
2057 	../Final/IRI/Year6/External/yogurt/Delivery_Stores 	8
2138 	../Final/IRI/Year7/External/yogurt/Delivery_Stores 	9
2067 	../Final/IRI/Year8/External/yogurt/Delivery_Stores 	10
2066 	../Final/IRI/Year9/External/yogurt/Delivery_Stores 	11

Distinct Headers found: 1 
['IRI_KEY OU EST_ACV  Market_Name              Open Clsd MskdName']

Total rows: 22658
Total files: 11


## Load into Dataframes

In [5]:
dfs = list()
for f in files:
    df = pd.read_fwf(f)
    df['yearid'] = int(f[f.find('/Year') + 5 : f.find('/External')])
    df['year'] = df[['yearid']] + 2000
    df['MskdName'] = df['MskdName'].apply(lambda x:  x.replace('Chain', '') if(pd.notnull(x)) else x)
    dfs.append((int(f[f.find('/Year') + 5 : f.find('/External')]), df))


### Unscramble MskdName

In [6]:
mask_path = !find $path -name masked_chain_xref1_11.csv

# use index as universal key
key = pd.read_csv(mask_path[0]).reset_index()

In [7]:
key.head()

,index,Year1,Year2,Year3,Year4,Year5,Year6,Year7,Year8,Year9,Year10,Year11
0,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,1,1
3,3,3,2,1,1,1,1,1,2,2,2,2
4,4,4,3,2,2,2,2,2,NaN,NaN,NaN,NaN


In [8]:
# convert type of columns for join
key = key.astype(str).applymap(lambda x: x.replace('.0', ''))
key.dtypes

index     object
Year1     object
Year2     object
Year3     object
Year4     object
Year5     object
Year6     object
Year7     object
Year8     object
Year9     object
Year10    object
Year11    object
dtype: object

In [9]:
# create new masked name that is uniform accross all years as 'name', drop original masked name
unmasked =[pd.merge(df, key[['index','Year' + str(y)]], 
          how='left', 
          left_on='MskdName', right_on='Year' + str(y))\
         .drop(['Year' + str(y), 'MskdName'],axis=1)\
         .rename(columns={'index':'name'})\
         for y, df in dfs]

In [17]:
stores = pd.concat(unmasked)
stores.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22658 entries, 0 to 2064
Data columns (total 9 columns):
IRI_KEY        22658 non-null int64
OU             22658 non-null object
EST_ACV        22658 non-null float64
Market_Name    22658 non-null object
Open           22658 non-null int64
Clsd           22658 non-null int64
yearid         22658 non-null int64
year           22658 non-null int64
name           22536 non-null object
dtypes: float64(1), int64(5), object(3)
memory usage: 1.7+ MB


In [18]:
stores.to_hdf('../Final/yogurt.h5', 'delivery_stores', complib='blosc', complevel=9)

In [20]:
test=pd.read_hdf('../Final/yogurt.h5', 'delivery_stores')
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22658 entries, 0 to 2064
Data columns (total 9 columns):
IRI_KEY        22658 non-null int64
OU             22658 non-null object
EST_ACV        22658 non-null float64
Market_Name    22658 non-null object
Open           22658 non-null int64
Clsd           22658 non-null int64
yearid         22658 non-null int64
year           22658 non-null int64
name           22536 non-null object
dtypes: float64(1), int64(5), object(3)
memory usage: 1.7+ MB
